In [1]:
import pandas as pd
import numpy
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,r2_score
import yfinance as yf

#Used Models
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [2]:
symbol='^NSEI'
ticker = yf.Ticker(symbol)
data=ticker.history(period='max',interval='1m')
data.reset_index(inplace=True)

In [3]:
data.head()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-04-05 10:42:00+05:30,22494.949219,22495.949219,22490.099609,22494.800781,0,0.0,0.0
1,2024-04-05 10:43:00+05:30,22494.800781,22495.400391,22491.849609,22492.550781,0,0.0,0.0
2,2024-04-05 10:44:00+05:30,22491.449219,22492.150391,22487.599609,22489.849609,0,0.0,0.0
3,2024-04-05 10:45:00+05:30,22493.300781,22496.800781,22493.250000,22494.099609,0,0.0,0.0
4,2024-04-05 10:46:00+05:30,22494.199219,22495.500000,22488.699219,22489.849609,0,0.0,0.0


In [4]:
data.tail()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
1492,2024-04-12 10:36:00+05:30,22656.900391,22657.650391,22653.050781,22657.199219,0,0.0,0.0
1493,2024-04-12 10:37:00+05:30,22657.199219,22661.849609,22657.050781,22661.000000,0,0.0,0.0
1494,2024-04-12 10:38:00+05:30,22661.199219,22664.500000,22659.849609,22663.800781,0,0.0,0.0
1495,2024-04-12 10:39:00+05:30,22664.050781,22665.000000,22657.199219,22658.900391,0,0.0,0.0
1496,2024-04-12 10:40:00+05:30,22659.599609,22661.199219,22653.050781,22657.800781,0,0.0,0.0


In [5]:
# data = pd.read_csv('/content/Nifty50_shortTerm_5m_New.csv')

print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1497 entries, 0 to 1496
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype                       
---  ------        --------------  -----                       
 0   Datetime      1497 non-null   datetime64[ns, Asia/Kolkata]
 1   Open          1497 non-null   float64                     
 2   High          1497 non-null   float64                     
 3   Low           1497 non-null   float64                     
 4   Close         1497 non-null   float64                     
 5   Volume        1497 non-null   int64                       
 6   Dividends     1497 non-null   float64                     
 7   Stock Splits  1497 non-null   float64                     
dtypes: datetime64[ns, Asia/Kolkata](1), float64(6), int64(1)
memory usage: 93.7 KB
None


In [6]:
print(data.describe())


               Open          High           Low         Close  Volume  \
count   1497.000000   1497.000000   1497.000000   1497.000000  1497.0   
mean   22650.112538  22653.125207  22646.665709  22649.938208     0.0   
std       82.610062     82.725652     82.540261     82.653328     0.0   
min    22477.449219  22483.599609  22473.050781  22476.699219     0.0   
25%    22621.650391  22623.599609  22618.000000  22621.750000     0.0   
50%    22669.949219  22672.550781  22667.449219  22669.650391     0.0   
75%    22707.949219  22712.050781  22704.699219  22707.800781     0.0   
max    22772.699219  22775.699219  22767.050781  22772.699219     0.0   

       Dividends  Stock Splits  
count     1497.0        1497.0  
mean         0.0           0.0  
std          0.0           0.0  
min          0.0           0.0  
25%          0.0           0.0  
50%          0.0           0.0  
75%          0.0           0.0  
max          0.0           0.0  


In [7]:
data.drop_duplicates()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-04-05 10:42:00+05:30,22494.949219,22495.949219,22490.099609,22494.800781,0,0.0,0.0
1,2024-04-05 10:43:00+05:30,22494.800781,22495.400391,22491.849609,22492.550781,0,0.0,0.0
2,2024-04-05 10:44:00+05:30,22491.449219,22492.150391,22487.599609,22489.849609,0,0.0,0.0
3,2024-04-05 10:45:00+05:30,22493.300781,22496.800781,22493.250000,22494.099609,0,0.0,0.0
4,2024-04-05 10:46:00+05:30,22494.199219,22495.500000,22488.699219,22489.849609,0,0.0,0.0
...,...,...,...,...,...,...,...,...
1492,2024-04-12 10:36:00+05:30,22656.900391,22657.650391,22653.050781,22657.199219,0,0.0,0.0
1493,2024-04-12 10:37:00+05:30,22657.199219,22661.849609,22657.050781,22661.000000,0,0.0,0.0
1494,2024-04-12 10:38:00+05:30,22661.199219,22664.500000,22659.849609,22663.800781,0,0.0,0.0
1495,2024-04-12 10:39:00+05:30,22664.050781,22665.000000,22657.199219,22658.900391,0,0.0,0.0


In [8]:
data.isnull().sum()

Datetime        0
Open            0
High            0
Low             0
Close           0
Volume          0
Dividends       0
Stock Splits    0
dtype: int64

In [9]:
data.drop(['Datetime','Volume','Dividends','Stock Splits'],axis=1,inplace=True)
data.head(10)



,Open,High,Low,Close
0,22494.949219,22495.949219,22490.099609,22494.800781
1,22494.800781,22495.400391,22491.849609,22492.550781
2,22491.449219,22492.150391,22487.599609,22489.849609
3,22493.300781,22496.800781,22493.250000,22494.099609
4,22494.199219,22495.500000,22488.699219,22489.849609
5,22489.900391,22491.349609,22488.250000,22490.900391
6,22490.900391,22493.349609,22487.900391,22489.349609
7,22489.050781,22491.500000,22486.800781,22489.849609
8,22490.349609,22495.199219,22489.400391,22494.000000
9,22495.849609,22502.050781,22495.150391,22500.949219


In [10]:
data['shift_close'] = data['Close'].shift(-1)
data.dropna(inplace=True)

In [11]:
data

,Open,High,Low,Close,shift_close
0,22494.949219,22495.949219,22490.099609,22494.800781,22492.550781
1,22494.800781,22495.400391,22491.849609,22492.550781,22489.849609
2,22491.449219,22492.150391,22487.599609,22489.849609,22494.099609
3,22493.300781,22496.800781,22493.250000,22494.099609,22489.849609
4,22494.199219,22495.500000,22488.699219,22489.849609,22490.900391
...,...,...,...,...,...
1491,22653.599609,22656.900391,22649.699219,22656.900391,22657.199219
1492,22656.900391,22657.650391,22653.050781,22657.199219,22661.000000
1493,22657.199219,22661.849609,22657.050781,22661.000000,22663.800781
1494,22661.199219,22664.500000,22659.849609,22663.800781,22658.900391


In [12]:
data.tail()

,Open,High,Low,Close,shift_close
1491,22653.599609,22656.900391,22649.699219,22656.900391,22657.199219
1492,22656.900391,22657.650391,22653.050781,22657.199219,22661.000000
1493,22657.199219,22661.849609,22657.050781,22661.000000,22663.800781
1494,22661.199219,22664.500000,22659.849609,22663.800781,22658.900391
1495,22664.050781,22665.000000,22657.199219,22658.900391,22657.800781


In [13]:
X=data.drop('shift_close',axis=1)
y = data['shift_close']

In [14]:
xy=pd.concat([X,y],axis=1)
xy.tail()

,Open,High,Low,Close,shift_close
1491,22653.599609,22656.900391,22649.699219,22656.900391,22657.199219
1492,22656.900391,22657.650391,22653.050781,22657.199219,22661.000000
1493,22657.199219,22661.849609,22657.050781,22661.000000,22663.800781
1494,22661.199219,22664.500000,22659.849609,22663.800781,22658.900391
1495,22664.050781,22665.000000,22657.199219,22658.900391,22657.800781


In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [16]:

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

predictions = my_model.predict(X_test)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_test)))

print("R2 Score" + str(r2_score(y_test,predictions)))


print("Accuracy Score : "+str(my_model.score(X_test,y_test)))

predictions


Mean Absolute Error: 4.308274739583333
R2 Score0.9939821798338968
Accuracy Score : 0.9939821798338968


array([22699.262, 22617.201, 22742.09 , 22495.045, 22739.63 , 22717.117,
       22669.984, 22645.635, 22695.834, 22606.863, 22699.89 , 22499.836,
       22713.568, 22718.102, 22742.682, 22671.154, 22668.102, 22655.162,
       22733.6  , 22664.594, 22665.098, 22676.912, 22707.256, 22742.654,
       22520.535, 22698.754, 22642.527, 22513.098, 22645.908, 22641.258,
       22517.74 , 22604.412, 22635.61 , 22716.139, 22717.605, 22701.129,
       22483.014, 22493.814, 22551.98 , 22596.086, 22718.422, 22677.236,
       22717.72 , 22495.232, 22497.37 , 22629.023, 22643.096, 22751.256,
       22745.148, 22706.709, 22493.195, 22745.861, 22655.525, 22675.012,
       22729.133, 22744.201, 22495.785, 22504.05 , 22711.645, 22504.348,
       22493.5  , 22505.133, 22627.645, 22744.285, 22626.432, 22713.229,
       22675.193, 22645.441, 22615.691, 22505.197, 22708.982, 22652.19 ,
       22731.146, 22659.934, 22749.424, 22679.195, 22752.242, 22713.568,
       22700.174, 22698.748, 22652.113, 22686.746, 

In [17]:
RandomeForestModel = RandomForestRegressor()
RandomeForestModel.fit(X_train, y_train)


predictions = RandomeForestModel.predict(X_test)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_test)))

print("R2 Score" + str(r2_score(y_test,predictions)))


print("Accuracy Score : "+str(RandomeForestModel.score(X_test,y_test)))

predictions

Mean Absolute Error: 4.07808632812504
R2 Score0.9950569848155277
Accuracy Score : 0.9950569848155277


array([22701.87748047, 22617.035     , 22741.60789063, 22495.91181641,
       22741.82783203, 22718.68173828, 22670.92919922, 22647.34408203,
       22696.62529297, 22604.8509375 , 22702.24751953, 22498.22246094,
       22710.82205078, 22717.28714844, 22743.28957031, 22672.139375  ,
       22669.24835938, 22655.38869141, 22735.57474609, 22667.45376953,
       22662.25978516, 22675.57644531, 22710.42138672, 22739.94203125,
       22518.03796875, 22699.82466797, 22645.49251953, 22513.18355469,
       22643.23396484, 22642.05417969, 22518.55808594, 22604.60732422,
       22638.51509766, 22717.893125  , 22718.24482422, 22703.05351563,
       22484.13078125, 22494.03466797, 22529.21050781, 22606.22328125,
       22716.81785156, 22676.86050781, 22720.96089844, 22496.13699219,
       22496.21628906, 22626.11421875, 22645.13332031, 22750.69125   ,
       22745.53068359, 22710.19490234, 22494.12482422, 22746.04236328,
       22656.02138672, 22679.05142578, 22729.51623047, 22743.15886719,
      

In [18]:
DesitionTreeModel = DecisionTreeRegressor()
DesitionTreeModel.fit(X_train, y_train)


predictions = DesitionTreeModel.predict(X_test)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_test)))

print("R2 Score" + str(r2_score(y_test,predictions)))


print("Accuracy Score : "+str(DesitionTreeModel.score(X_test,y_test)))

predictions

Mean Absolute Error: 5.011295572916667
R2 Score0.9933393078847328
Accuracy Score : 0.9933393078847328


array([22701.84960938, 22623.75      , 22740.55078125, 22494.55078125,
       22746.94921875, 22717.19921875, 22670.19921875, 22649.69921875,
       22701.44921875, 22603.65039062, 22704.90039062, 22501.25      ,
       22704.30078125, 22715.94921875, 22742.25      , 22670.19921875,
       22673.40039062, 22655.90039062, 22739.05078125, 22667.34960938,
       22662.59960938, 22669.84960938, 22709.80078125, 22732.84960938,
       22515.44921875, 22702.25      , 22635.69921875, 22513.94921875,
       22644.15039062, 22641.84960938, 22520.05078125, 22606.80078125,
       22634.25      , 22719.44921875, 22713.30078125, 22701.5       ,
       22481.25      , 22496.40039062, 22524.80078125, 22603.65039062,
       22711.5       , 22676.94921875, 22718.5       , 22493.19921875,
       22496.55078125, 22624.90039062, 22643.75      , 22750.80078125,
       22744.44921875, 22713.90039062, 22496.94921875, 22748.19921875,
       22651.75      , 22683.30078125, 22727.5       , 22746.15039062,
      

In [19]:

import pickle
pickle.dump(my_model,open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/XGBoost.pkl','wb'))
pickle.dump(RandomeForestModel,open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/RandomForestRegressor.pkl','wb'))
pickle.dump(DesitionTreeModel,open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/DesicionTreeModel.pkl','wb'))

# pickle.dump(my_model,open('/content/drive/MyDrive/AI COLLAGE/5m_Nifty_RandomForestRegressor.pkl','wb'))


import yfinance as yf
import datetime as dt
import pickle
from sklearn.metrics import mean_absolute_error,r2_score,accuracy


# Load the model from the pickle file
with open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/RandomForestRegressor.pkl', 'rb') as f:
    Rand = pickle.load(f)


with open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/DesicionTreeModel.pkl', 'rb') as f:
    Decision = pickle.load(f)


with open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/XGBoost.pkl', 'rb') as f:
    XgBoostModel = pickle.load(f)



while True:
    ticker = yf.Ticker('^NSEI')
    start=dt.datetime.now().strftime('%Y-%m-%d')
    end=(dt.datetime.now() + dt.timedelta(days=1)).strftime('%Y-%m-%d')

    data = ticker.history(interval='5m',start=start,end=end)
    data.reset_index(inplace=True)
    # print(data)
    data.columns = data.columns.str.strip()  # Remove extra spaces from column names

    data.drop(['Datetime', 'Volume', 'Dividends', 'Stock Splits'], inplace=True, axis=1)
    y = data['Close'].copy()
    data.drop(['Close'], inplace=True, axis=1)

    print("Random Forest Regressor")
    print("---------------------------\n")
    Rand_predic=Rand.predict(data)
    last_pred=Rand_predic[-1]
    print(f'Last Prediction ----------> :{ last_pred}')
    print(f'mae : {mean_absolute_error(y,Rand_predic)}')
    # print(f"Rand Accuracy Score :{}")
    print(f'r2 : {r2_score(y,Rand_predic)}')
    print("---------------------------")
    print("\n\n\n")

    print("Desicion Tree Regressor")
    print("---------------------------\n")
    DecisionPred=Decision.predict(data)
    last_pred=DecisionPred[-1]
    mae=mean_absolute_error(y,DecisionPred)
    # last_pred=last_pred+mae
    print(f'Last Prediction ----------> :{ last_pred}')
    print(f'mae : {mae}')
    print(f'r2 : {r2_score(y,DecisionPred)}')
    print("---------------------------")
    print("\n\n\n")

    print("XgBoostModel ")
    print("---------------------------\n")
    XgBoostPred=XgBoostModel.predict(data)
    last_pred=XgBoostPred[-1]
    print(f'Last Prediction ----------> :{ last_pred}')
    print(f'mae : {mean_absolute_error(y,XgBoostPred)}')
    print(f'r2 : {r2_score(y,XgBoostPred)}')
    print("---------------------------")
    print("\n\n\n")
    # data